In [ ]:
from math import sqrt, log
import glob
import re

from utils import Graph
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
import numpy as np

T = 87       # K
n = 2.11e22  # cm^-3

def mobility(E):
    a0 = 551.6
    a1 = 7953.7
    a2 = 4440.43
    a3 = 4.29
    a4 = 43.63
    a5 = 0.2053
    T0 = 89

    num = a0 + a1*E + a2*E**(3/2) + a3*E**(5/2)
    den = 1 + (a1/a0)*E + a4*E**2 + a5*E**3

    return (num / den) * (T / T0)**(-3/2)

mobility_v = np.vectorize(mobility)

In [ ]:
T0 = 90.371
P1 = -0.01481
P2 = -0.0075
P3 = 0.141
P4 = 12.4
P5 = 1.627
P6 = 0.317

def DriftVelWalkowiak(efield,T=89.):
    a = P1*(T-T0)+1
    b = (P3 * efield * np.log(1+P4/efield)+P5*(efield**P6))
    c = P2 * ( (T-T0))
    f = a*b+c
    return f * (1e3)
    #return a*b+c

DriftVelWalkowiak_v = np.vectorize(DriftVelWalkowiak)

In [ ]:
efield_v = np.linspace(10,12500,1000)
efield_v_w = np.linspace(500,12500,1000)
driftwalk_v = DriftVelWalkowiak_v(efield_v_w/1e3)
mob_v = mobility_v(efield_v * 1e-3) * (efield_v * 1e-2)

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.plot(efield_v_w,driftwalk_v)
plt.plot(efield_v,mob_v)
plt.xscale('log')
plt.yscale('log')

plt.show()

In [ ]:
file_tree = {}

#field_s = [30,60,125,250,500,1000,2000,3500,5000,7500,10000,12500,15000,17500,20000]
field_s = [10,30,50,100,200,500,1000,2500,5000,7500,15000]

for field in field_s:
    #for file in glob.glob(f'lar-drift-velocity/{field}/*.txt'):
    for file in glob.glob(f'driftvel/{field}/*.txt'):
    #for file in glob.glob(f'diff/driftvel/{field}/*.txt'):
        key = int(re.search(r'(\d*)(?:V)', file).group(1))
        if (key in file_tree):
            file_tree[key].append(file)
        else:
            file_tree[key] = [file]

The below plots drift velocity of the simulated electron cloud versus time. The data lines are taken from Li at https://lar.bnl.gov/properties/trans.html. For each bulk field strength, $100$ electrons were spawned and allowed to propagate for $10$ ns.

In [ ]:
eVs_all = []
bincentres = 0

simulation_v = []

for key, value in sorted(file_tree.items()):
    print(key)
    group = Graph(value,nvar=10)
    #time, vel = self.t, self.drift
    time, vel = group.t, group.drift
    ke = group.ke
    
    # plt.rcParams['font.size'] = 14
    # plt.rcParams['font.family'] = 'serif'
    # fig = plt.figure(figsize=(6, 6))
    
    plt.title(f'Electron Drift Velocities at {key} V/cm')
    plt.xlabel('Time [ns]')
    plt.ylabel('Drift Velocity [m/s]')
    
    for i, t in enumerate(time):
        plt.plot(t, vel[i], alpha=0.6)
        
    MIN = 0
    MAX = 300
    
    theory = mobility(key * 1e-3) * (key * 1e-2)
    
    driftvel = group.drift_mean()
    #theory_v.append(theory)
    
    plt.plot(time[0], np.full(len(time[0]), driftvel), 'b--', label='Mean Drift Velocity')
    plt.plot([MIN,MAX], [theory, theory], 'r--', label='Data from [6]')
    plt.xlim(MIN,MAX)
    plt.ylim(0,theory*2)
    #plt.ylim(-100, 10000)
    
    simulation_v.append(driftvel)
    
    max_ev = np.amax([np.amax(k) for k in ke])
    
    bin_list = np.linspace(0, 2.6, 100)
    H, bins = np.histogram(ke[0], bins=bin_list, density=True)
    
    for i in range(group.n - 1):
        H_, bins_ = np.histogram(ke[i + 1], bins=bin_list, density=True)
        H += H_
        
        
    bincentres = [(bins[i] + bins[i + 1]) / 2. for i in range(len(bins) - 1)]
    eVs_all.append(H)

    plt.grid()
    plt.legend()
    plt.show()
    
    # plt.savefig(f'lar_drift_vel_{key}.pdf', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(6,6))

plt.plot(efield_v_w,driftwalk_v,label='Walkoviak \'99')
plt.plot(efield_v,mob_v)

plt.plot(field_s,simulation_v,'ko')

plt.legend(loc=2,fontsize=14)
plt.xscale('log')
plt.xlabel('Electric Field [V/cm]',fontsize=16)
plt.ylabel('Drift Velocity [m/s]',fontsize=16)
plt.yscale('log')
#plt.plot(efield_v,mob_v)

#plt.xlim([0,600])
#plt.ylim([0,2000])

plt.show()

The below plots projections of the trajectories of the simulations.

In [ ]:
for key, value in sorted(file_tree.items()):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
    
    group = Graph(value,nvar=10)
    
    for i in range(group.n):
        if (i%10 != 0): continue
        x, y = group.x[i], group.y[i]
        ax1.plot(x, y)
        
    ax1.set_xlabel('x [$\mu$m]')
    ax1.set_ylabel('y [$\mu$m]')
    ax1.set_title(f'Electron Trajectories at {key} V/cm')
    
    ax1.grid()
    
    for i in range(group.n):
        if (i%10 != 0): continue
        x, z = group.x[i], group.z[i]
        ax2.plot(x, z)
        
    ax2.set_xlabel('x [$\mu$m]')
    ax2.set_ylabel('z [$\mu$m]')
    ax2.set_title(f'Electron Trajectories at {key} V/cm')
    
    ax2.grid()
    
    for i in range(group.n):
        if (i%10 != 0): continue
        x, t = group.y[i], group.t[i]
        ax3.plot(t, x)
        
    ax3.set_ylabel('x [$\mu$m]')
    ax3.set_xlabel('t [ns]')
    ax3.set_title(f'Electron Trajectories at {key} V/cm')
    ax3.grid()

    plt.tight_layout()
    plt.show()

In [ ]:
plt.rcParams.update({'font.size': 16})

The below plots simulated mobility against data. The data from Li et al. is found at https://lar.bnl.gov/properties/trans.html, while the data from Wojcik et al. is found at https://doi.org/10.1088/0022-3727/19/12/011.

In [ ]:
volts = []
means = []
std_devs = []

# plt.rcParams['font.size'] = 14
# plt.rcParams['font.family'] = 'serif'
# fig = plt.figure(figsize=(6, 6))

# From https://lar.bnl.gov/properties/trans.html
c = np.logspace(-21, -18, 10)
d = mobility(c * n * 1e-3) * n

# From https://doi.org/10.1088/0022-3727/19/12/011
a = [1.4e-21, 2.8e-21, 6.2e-21, 1.2e-20, 2.5e-20, 5.1e-20, 1e-19, 2.2e-19, 4.6e-19, 1e-18]
b = [9e24, 10e24, 10e24, 9e24, 6e24, 4.4e24, 3e24, 2e24, 1e24, 7e23]

#Wolkowiak
fig = plt.figure(figsize=(8,6))

plt.plot(a, b, 'k--', label='J. Phys. D: Appl. Phys. 19 2309')
plt.errorbar(c, d, color='m', ls='--', yerr=(0.2 * d), label='lar.bnl.gov/properties/trans.html')

for key, value in sorted(file_tree.items()):
    graph = Graph(value,nvar=10)
    volts.append(key)
    means.append(graph.drift_mean())
    std_devs.append(graph.drift_std_dev())

x, y, err = [], [], []

for j, mu in enumerate(means):
    x.append(float(volts[j]) / n)
    y.append(means[j] * (n / float(volts[j])) * 1e2)
    err.append(std_devs[j] * (n / float(volts[j])) * 1e2)

plt.errorbar(x, np.array(y), yerr=np.array(err), label=f'TRANSLATE simulation')

plt.xlabel('Reduced Electric Field, E/n [Vcm$^2$]')
plt.ylabel('Mobility, $\mu$n [cmV/s]')
plt.title('Electron Mobility in LAr')

plt.xscale('log')
plt.yscale('log')

plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('plots/lar_mobility.pdf', dpi=250)
plt.show()
